In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import keras
keras.__version__

'2.4.3'

In [ ]:
import tensorflow
tensorflow.__version__

'2.3.0'

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Aug 27 03:06:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    49W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.7 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [ ]:
from tensorflow import TensorSpec, float32, int32
from tensorflow import keras, data
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.data.experimental import load

In [ ]:
# try to load tensorflow datasets from google drive into this notebook
# set element spec
elem_spec = (TensorSpec(shape=(None, 256, 256, 3), dtype=float32, name=None), TensorSpec(shape=(None,), dtype=int32, name=None))
# path to files:
X_train_data_path = '/content/drive/My Drive/TF_datasets/X_train'
X_test_data_path = '/content/drive/My Drive/TF_datasets/X_test'
X_train = load(X_train_data_path, element_spec=elem_spec, compression='GZIP', reader_func=None)
X_test = load(X_test_data_path, element_spec=elem_spec, compression='GZIP', reader_func=None)

In [ ]:
num_classes = 2
epochs = 500 # number of passes through the entire train dataset before weights "final"
AUTOTUNE = data.experimental.AUTOTUNE
img_height = 256
img_width = 256
nb_filters = 32    # number of convolutional filters to use - want 1 for each "feature" you think exists in images if more, more parameters
pool_size = (2, 2)  # pooling decreases image size, reduces computation, adds translational invariance
kernel_size = (2, 2)  # convolutional kernel size, slides over image to learn features
X_train = X_train.cache().shuffle(32).prefetch(buffer_size=AUTOTUNE) # include shuffle? set to 32? X_train.cache().shuffle(1000)
# Note: While large buffer_sizes shuffle more thoroughly, they can take a lot of memory, and significant time to fill. 
# It will shuffle your entire dataset (x, y and sample_weight together) first and then make batches according to the batch_size argument you passed to fit.
# Consider using Dataset.interleave across files if this becomes a problem.
X_test = X_test.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 3)),
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.Conv2D(nb_filters, (kernel_size[0], kernel_size[1]), padding='same', activation='relu'), # was 16, 32, 64
    layers.MaxPooling2D(pool_size=pool_size),
    layers.Conv2D(nb_filters*2, (kernel_size[0], kernel_size[1]), padding='same', activation='relu'), # drop layers.. for initial testing
    layers.MaxPooling2D(pool_size=pool_size),
    layers.Conv2D(nb_filters*3, (kernel_size[0], kernel_size[1]), padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=pool_size),
    layers.Conv2D(nb_filters*4, (kernel_size[0], kernel_size[1]), padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=pool_size),
    # insert more layers?
    # insert callback?
    layers.Flatten(),
    layers.Dense(256, activation='relu'), # increase this? add another dense layer?
    layers.Dropout(0.5),
    #layers.Dense(num_classes, activation='relu') # or sigmoid for binary? relu?
    layers.Dense(1, activation='sigmoid') # or sigmoid for binary? relu?
])

In [ ]:
model.compile(optimizer='adam', # adadelta sgd
              loss=keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 256, 256, 3)       0         
_________________________________________________________________
random_flip (RandomFlip)     (None, 256, 256, 3)       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 256, 256, 3)       0         
_________________________________________________________________
random_zoom (RandomZoom)     (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 32)      416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      8

In [ ]:
history = model.fit(
            X_train,
            validation_data = X_test,
            epochs = epochs,
            verbose = 2
)

Epoch 1/500
406/406 - 133s - loss: 0.6488 - accuracy: 0.6488 - val_loss: 0.5988 - val_accuracy: 0.7021
Epoch 2/500
406/406 - 17s - loss: 0.5751 - accuracy: 0.7112 - val_loss: 0.5237 - val_accuracy: 0.7594
Epoch 3/500
406/406 - 17s - loss: 0.5390 - accuracy: 0.7413 - val_loss: 0.5160 - val_accuracy: 0.7631
Epoch 4/500
406/406 - 16s - loss: 0.5192 - accuracy: 0.7498 - val_loss: 0.5180 - val_accuracy: 0.7579
Epoch 5/500
406/406 - 16s - loss: 0.5158 - accuracy: 0.7573 - val_loss: 0.5098 - val_accuracy: 0.7619
Epoch 6/500
406/406 - 16s - loss: 0.5080 - accuracy: 0.7599 - val_loss: 0.4862 - val_accuracy: 0.7714
Epoch 7/500
406/406 - 16s - loss: 0.4997 - accuracy: 0.7645 - val_loss: 0.5293 - val_accuracy: 0.7665
Epoch 8/500
406/406 - 17s - loss: 0.4917 - accuracy: 0.7684 - val_loss: 0.5386 - val_accuracy: 0.7637
Epoch 9/500
406/406 - 17s - loss: 0.4843 - accuracy: 0.7741 - val_loss: 0.5071 - val_accuracy: 0.7665
Epoch 10/500
406/406 - 16s - loss: 0.4753 - accuracy: 0.7832 - val_loss: 0.4750 -

In [ ]:
score = model.evaluate(X_test, verbose=1) 
print('Test score:', score[0])
print('Test accuracy:', score[1])

102/102 [==============================] - 1s 13ms/step - loss: 0.7683 - accuracy: 0.8786
Test score: 0.7682643532752991
Test accuracy: 0.8786198496818542


In [ ]:
# without re-running notebook, can load the best model to do predictions and find incorrect predictions
model = keras.models.load_model('/content/drive/My Drive/TF_models/500_epochs_model_wild_est_binary')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 256, 256, 3)       0         
_________________________________________________________________
random_flip (RandomFlip)     (None, 256, 256, 3)       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 256, 256, 3)       0         
_________________________________________________________________
random_zoom (RandomZoom)     (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 32)      416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      8

In [ ]:
# save model
# model.save('/content/drive/My Drive/TF_models/500_epochs_model_wild_est_binary')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/TF_models/500_epochs_model_wild_est_binary/assets
